<img src="assets/img/chm-banner.jpg">

# Estimating canopy height with RasterFlow

This notebook will guide you through estimating canopy height from aerial imagery, using Wherobots RasterFlow and the Canopy Height Model. You will gain a hands-on understanding of how to run models like the Meta and World Resource Institute's Canopy Height prediction model on your selected area of interest.

## What you will learn

This notebook will teach you to:

* Use RasterFlow to run GeoAI models like the Meta and World Resource Institute's Canopy Height prediction model and generate inference results

## Wherobots RasterFlow

With Wherobots RasterFlow, developers can use GeoAI models to generate insights at scale from satellite and aerial imagery.

RasterFlow has built-in open models for common use-cases, including the Canopy Height Model (CHM) for predicting the height of the tree canopy.

### Meta and World Resource Institute's Canopy Height prediction model (Meta CHM v1)

The [Meta and World Resource Institute's Canopy Height prediction model (Meta CHM v1)](https://github.com/facebookresearch/HighResCanopyHeight/)<sup>1</sup> model is an example of an open regression model that can predict the height of the tree canopy from high resolution imagery.

This model generates a raster where each pixel is the estimated tree canopy height.

It was trained on high-resoluton imagery ([Maxar Vivid2 mosaic](https://pro-docs.maxar.com/en-us/VividMosaics/VividMosaics_intro.htm) with 0.5m resolution), existing labeled canopy height maps and airborne laser scans (LIDAR).

## Selecting an Area of Interest (AOI)
We will choose an Area of Interest (AOI) for our analysis. The area around Nashua, NH has a combination of urban settings, parks and forests so we will try out the model there.

In [ ]:
import wkls
import geopandas as gpd
import os

# Generate a geometry for Nashua, NH using WKLS (https://github.com/wherobots/wkls)
gdf = gpd.read_file(wkls.us.nh.nashua.geojson())

# Save the geometry to a parquet file in the user's S3 path
aoi_path = os.getenv("USER_S3_PATH") + "nashua.parquet"
gdf.to_parquet(aoi_path)

## Initializing the RasterFlow client

In [ ]:
from datetime import datetime

from rasterflow_remote import RasterflowClient

from rasterflow_remote.data_models import (
    ModelRecipes
)

rf_client = RasterflowClient()

## Running a model

RasterFlow has pre-defined workflows to simplify orchestration of the processing steps for model inference.  These steps include:
* Ingesting imagery for the specified Area of Interest (AOI)
* Generating a seamless image from multiple image tiles (a mosaic) 
* Running inference with the selected model

The output is a Zarr store of the model outputs.

Note: This step will take approximately 20 minutes to complete.

In [ ]:
model_outputs = rf_client.build_and_predict_mosaic_recipe(
    # Path to our AOI in GeoParquet or GeoJSON format
    aoi = aoi_path,

    # Date range for imagery to be used by the model
    start = datetime(2023, 1, 1),
    end = datetime(2024, 1, 1),

    # Coordinate Reference System EPSG code for the output mosaic   
    crs_epsg = 3857,

    # The model recipe to be used for inference (FTW in this case)
    model_recipe = ModelRecipes.META_CHM_V1
)

print(model_outputs)

## Visualize a subset of the model outputs
We will use hvplot and datashader to visualize a small subset of the model outputs.

In [ ]:
# Import libraries for visualization and coordinate transformation
import hvplot.xarray
import xarray as xr
import s3fs 
import zarr
from pyproj import CRS, Transformer
from holoviews.element.tiles import EsriImagery 

# Select the field_boundaries class from the model outputs to visualize
class_to_visualize = "height"

# Open the Zarr store and select the field_boundaries band
fs = s3fs.S3FileSystem(profile = "default", asynchronous=True)
zstore = zarr.storage.FsspecStore(fs, path=model_outputs[5:])
ds = xr.open_zarr(zstore).sel(band=class_to_visualize)

# Define the bounding box coordinates for an area in Nashua, NH
min_lon = -71.56  
max_lon = -71.54   
min_lat = 42.77  
max_lat = 42.79  

# Set up coordinate reference systems for transformation
source_crs = CRS.from_epsg(4326)  # WGS84 (lat/lon)
target_crs = CRS.from_epsg(3857)  # Web Mercator (meters)

# Create a transformer to convert from lat/lon to meters
transformer = Transformer.from_crs(source_crs, target_crs, always_xy=True)

# Transform the bounding box coordinates from lat/lon to meters
x_coords = [min_lon, max_lon]
y_coords = [min_lat, max_lat]

x_meters, y_meters = transformer.transform(x_coords, y_coords)

# Extract the min/max values for slicing the dataset
x_slice_min = min(x_meters)
x_slice_max = max(x_meters)
y_slice_min = min(y_meters)
y_slice_max = max(y_meters)

# Slice the dataset to the bounding box (note: y is reversed for correct orientation)
ds_subset = ds.sel(
    x=slice(x_slice_min, x_slice_max),
    y=slice(y_slice_max, y_slice_min) 
)

# Select the first time step and extract the variables array
arr_subset = ds_subset.isel(time=0)["variables"]

# Create a base map layer using Esri satellite imagery
base_map = EsriImagery()

# Create an overlay layer from the model outputs with hvplot
output_layer = arr_subset.hvplot(
    x = "x",
    y = "y",
    geo = True,           # Enable geographic plotting
    dynamic = True,       # Enable dynamic rendering for interactivity
    rasterize = True,     # Use datashader for efficient rendering of large datasets
    cmap = "viridis",     # Color map for visualization
    aspect = "equal",     # Maintain equal aspect ratio
    title = "CHM Model Outputs" 
).opts(
    width = 600, 
    height = 600,
    alpha = 0.7           # Set transparency to see the base map underneath
)

# Combine the base map and output layer
final_plot = base_map * output_layer
final_plot

### References

1. **Tolan, J., Yang, H.-I., Nosarzewski, B., Couairon, G., Vo, H. V., Brandt, J., Spore, J., Majumdar, S., Haziza, D., Vamaraju, J., et al. (2024).** Very high resolution canopy height maps from RGB imagery using self-supervised vision transformer and convolutional decoder trained on aerial lidar. *Remote Sensing of Environment*, 300, 113888.